## Concise: 1: Start and mesh
Author: yanjun zhang, Source from book "Abali - 2017 - Computational Reality" P119

In [ ]:
from disc_f import *

# mesh-size, contact area coefficient
mesh_min, mesh_max = 3, 5
c_contact          = 1
# Each time step rotation angular, and acc during lag, 1 is full acc, 0 is no acc.
angular_r          = 224
v_vehicle, c_acc   = 160, 1
z1,z2,z3,z_all     = 20,33,30,8
pad_v_tag          = 32
# calling local functions to get all parameters
(dt, P, g, num_steps, h, radiation, v_angular, Ti, Tm, S_rub_circle, t, rho, c, k, t_brake, 
S_total,) = vehicle_initial (angular_r, v_vehicle, c_contact, c_acc)
print("1: Total braking tims is ", round(sum(dt), 2), "s")
print("2: Total numb steps is ", num_steps)

## here use lots of abbreviation, details are in disc_f
domain, cell_markers, facet_amrkers, mesh_name, mesh_name1, mesh_name2 \
                       = mesh_brake_all(mesh_min,mesh_max,pad_v_tag)
V, T_init, u_n         = initial_u_n(domain, Ti)
fdim, bc, mesh_brake, all_e,xdmf, x_co, y_co, ds, b_con \
                       = mesh_setup(domain, V,mesh_name1,num_steps, \
                         angular_r, mesh_name2, c_contact,z_all,Tm, S_rub_circle)
# Initialize
problem,u,v,f,n_vector = variation_initial(V, T_init,domain, rho, c, b_con,\
                          radiation, h, k, xdmf,dt,ds,u_n, Tm,g,bc);
n,converged = solver_setup_solve(problem,u)
## Visualization of time dependent problem using pyvista
gif_name    = "T-s-{}-d-{}-{}-c-{}-e-{}.gif".format(num_steps, angular_r, mesh_name2, c_contact, all_e)
plotter, sargs, renderer, warped, viridis, grid = plot_gif(V,u,gif_name)
##solve
num_steps= int(num_steps/20)
T_array     = solve_heat(Ti, u, num_steps, dt, x_co, y_co, angular_r,t_brake, \
                     domain,S_rub_circle,fdim, rho, c, v, radiation, k, h, \
                     f,Tm,u_n,g, ds, xdmf, b_con,bc,plotter,warped)
csv_name    = "Result_T-s-{}-d-{}-{}-c-{}-e-{}.csv".format(num_steps, angular_r, mesh_name2, c_contact, all_e  )
# got the Temperature data
save_t_T(csv_name, T_array)

from IPython.display import display, Image
display(Image(gif_name))

# 2: New brake pad points, add boundary for rubbing elements

In [ ]:
## plot the temperature
mesh_n_pad = mesh_del_disc(mesh_name1, "new_pad.msh")
T_new_p, pad_node_coordinates  = T_pad_transfer1( mesh_name1, mesh_n_pad, u_n, mesh_brake, pad_v_tag )
domain_pad, cell_mark_pad, facet_mark_pad = gmshio.read_from_msh( mesh_n_pad , MPI.COMM_WORLD, 0, gdim=3 )
plot_T_pad( domain_pad, T_new_p).show()

# defin the pad domain
VT      = fem.functionspace(domain_pad, ("CG", 1))         #define the finite element function space
Delta_T = fem.Function(VT, name ="Temperature_variation")  # T_ is the test function, like v
for i in range(len(T_new_p)):
    Delta_T.vector.array[i] = T_new_p[i]

#######try to make domain only for brake pad.
E    = fem.Constant(domain_pad, 50e3)             # Elastic module
nu   = fem.Constant(domain_pad, 0.2)              # Poission ratio
gdim = domain_pad.geometry.dim

mu    = E / 2 / (1 + nu)                          # Shear modulus
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)          # Lame parameters
alpha = fem.Constant(domain_pad, 1e-5)            # Thermal expansion coefficient
f1    = fem.Constant(domain_pad, (0.0, 0.0, 0.0)) # O for external force

def eps(v):                                       # epsilon, strain, the deforamtion, dy/y 
    return ufl.sym(ufl.grad(v))
def sigma(v, Delta_T):                            # sigmathis is sigma
    return (lmbda * ufl.tr(eps(v)) - alpha * (3 * lmbda + 2 * mu) * Delta_T 
    ) * ufl.Identity(gdim)  + 2.0 * mu * eps(v)   # here braces is important, can not be in above line

Vu = fem.functionspace(domain_pad, ("CG", 1, (gdim,))) 
du = ufl.TrialFunction(Vu)
u_ = ufl.TestFunction(Vu)

Wint = ufl.inner(sigma(du, Delta_T), eps(u_)) * ufl.dx  # here du is unkown
aM   = ufl.lhs(Wint)                                    # Wint is long and lhs can help to distinguish unkown and know.
LM   = ufl.rhs(Wint) + ufl.inner(f1, u_) * ufl.dx       # knows parameters are in lhs

def up_side(x):
    return np.isclose(x[2], (z1+z2+z3))

up_dofs_u = fem.locate_dofs_geometrical(Vu, up_side)   # lateral sides of domain
bcu       = [fem.dirichletbc(np.zeros((gdim,)), up_dofs_u, Vu)]  # displacement Vu is fixed in lateral sides

u_d     = fem.Function(Vu, name="Displacement")
problem = fem.petsc.LinearProblem(aM, LM, u=u_d, bcs=bcu)
problem.solve()

scale_factor = 100
plot_s_pad = plot_S_pad(Vu,u_d,scale_factor )
plot_s_pad.show()


## have one booundary

## Plot contact information

In [ ]:
# Visualizing the displacement profile in the z-direction
plt.plot(u_d.x.array[2::3])
plt.title("Displacement in z-direction")
plt.xlabel("Node Index")
plt.ylabel("Displacement (z-direction)")
plt.grid()
plt.show()

# Identify contact areas and plot them
contact_nodes = np.where(u_d.x.array[2::3] >= z1)[0]  # Nodes in contact with the surface at z1
contact_coordinates = pad_node_coordinates[contact_nodes]  # Get the coordinates of contact nodes

# Plotting contact nodes
plt.scatter(contact_coordinates[:, 0], contact_coordinates[:, 1], label="Contact Nodes", c='red', s=10)
plt.title("Contact Area on Brake Pad")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.grid()
plt.legend()
plt.show()